In [21]:
import pandas as pd
import numpy as np

In [22]:
df = pd.read_csv(rf'D:\DS_Work\Projects\GemStone Price\notebooks\data\train.csv')
df.drop(columns=['id'], inplace=True)
df.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [23]:
X = df.drop(columns=['price'])
y = df[['price']]

In [ ]:
categorical_cols = X.columns[X.dtypes == 'O'] 
numerical_cols = X.columns[X.dtypes != 'O']  

In [39]:
cut_categories=['Fair', 'Good',  'Very Good', 'Premium', 'Ideal']
color_categories=['D','E', 'F', 'G', 'H', 'I', 'J']
clarity_categories= ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']

In [40]:
from sklearn.impute import SimpleImputer ##Handling Missing Values
from sklearn.preprocessing import StandardScaler ##Handling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder ##For Doing Ordinal Encoding

## Pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer 

In [ ]:
## Numerical Pipeline
num_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()) 
    ]
)

## Categorical Pipeline
cat_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder', OrdinalEncoder(categories=[cut_categories, color_categories, clarity_categories])),
        ('scaler', StandardScaler()) 
    ]
)

preprocessor = ColumnTransformer([
    ('num_pipeline', num_pipeline, numerical_cols),
    ('cat_pipeline', cat_pipeline, categorical_cols)
])

In [42]:
## Train Test Split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=30)

In [45]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train), columns=preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.transform(X_test), columns=preprocessor.get_feature_names_out())

In [47]:
## Model Training

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error


In [54]:
import numpy as np

def evaluate_model (true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mse)
    r2 = r2_score(true, predicted)

    return mae, rmse, r2

In [ ]:
## Train Multiple Models

models={
    'LinearRegression': LinearRegression(),
    'Lasso': Lasso(),
    'Ridge': Ridge(),
    'ElasticNet': ElasticNet()
}

trained_model_list = []
model_list = []
r2_list = []

for i in range(len(models)):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    #Make Predictions
    y_pred = model.predict(X_test)
    mae, rmse, r2 = evaluate_model(y_test, y_pred)

    print (list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    trained_model_list.append(list(models.values())[i])
    
    print ('Model Training Performance')
    print ('RMSE:',rmse)
    print ('MAE:',mae)
    print ('RMSE:',rmse)
    print ('R2 Score:',round(r2*100,2))

    r2_list.append(r2)
    print ('='*35)
    print ('\n')

LinearRegression
Model Training Performance
RMSE: 1013.9047094344004
MAE: 674.0255115796832
RMSE: 1013.9047094344004
R2 Score: 93.69


Lasso
Model Training Performance
RMSE: 1013.8784226767013
MAE: 675.0716923362161
RMSE: 1013.8784226767013
R2 Score: 93.69


Ridge
Model Training Performance
RMSE: 1013.9059272771647
MAE: 674.0555800798206
RMSE: 1013.9059272771647
R2 Score: 93.69


ElasticNet
Model Training Performance
RMSE: 1533.4162456064048
MAE: 1060.7368759154729
RMSE: 1533.4162456064048
R2 Score: 85.56


